In [10]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [2]:
# loading the trained model
pickle_rf = open('classifier_rf.pkl', 'rb') 
clf_rf = pickle.load(pickle_rf)

pickle_scaler = open('scaler_5.pkl', 'rb') 
scaler_5 = pickle.load(pickle_scaler)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
scaled_df = pd.read_csv('modeling_dataset_vScaled.csv', index_col=0)

In [4]:
target_alt = 'final_phys_diagn_alt'

filter_1 = scaled_df[target_alt] != 'srbd_multiple'
filter_2 = scaled_df[target_alt] != 'multiple'
filter_3 = scaled_df[target_alt] != 'other'

scaled_df = scaled_df[filter_1 & filter_2]
scaled_df[target_alt] = scaled_df[target_alt].where(scaled_df[target_alt] == 'srbd_apnea', 'non_apnea')

In [5]:
asq_imp = pd.read_csv('top30_lgbm.csv', index_col=0)
imp_feat = asq_imp.columns

In [6]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df[imp_feat], scaled_df[target_alt], test_size=0.2, random_state=2021)
undersample = RandomUnderSampler(random_state=2021)
X_train, y_train = undersample.fit_resample(X_train, y_train)
y_train.value_counts()

non_apnea     326
srbd_apnea    326
Name: final_phys_diagn_alt, dtype: int64

In [8]:
knn_pred = clf_rf.predict(X_test[imp_feat[:5]])
conf = pd.crosstab(y_test, knn_pred, rownames=['True'], colnames=['Predicted'], margins=True).T

In [14]:
conf.to_csv('rf_confusion.csv')